In [1]:
import os
import joblib
import pandas as pd
import json

# ตรวจสอบว่าไฟล์มีอยู่จริงก่อนโหลด
model_path = "machine_failure_model.pkl"
encoder_path = "label_encoder.pkl"

if os.path.exists(model_path) and os.path.exists(encoder_path):
    loaded_model = joblib.load(model_path)
    loaded_le = joblib.load(encoder_path)
    print("✅ โหลดโมเดลสำเร็จ")
else:
    print("❌ ไม่พบไฟล์โมเดล โปรดตรวจสอบเส้นทางไฟล์")
    exit()

def get_abnormal_details(input_data):
    """
    ฟังก์ชันนี้จะตรวจสอบค่าจาก input_data และคืนรายการของความผิดปกติพร้อมค่าที่ trigger
    """
    details = []
    if input_data["Temperature"] > 100:
        details.append(f"🔥 High Temperature ({input_data['Temperature']}°C)")
    if input_data["Vibration"] > 1.0:
        details.append(f"🔧 High Vibration ({input_data['Vibration']})")
    if input_data["Machine_Age"] > 7:
        details.append(f"⚙ Aging (Machine Age: {input_data['Machine_Age']} years)")
    if input_data["Operating_Hours"] > 5000:
        details.append(f"⏳ High Operating Hours ({input_data['Operating_Hours']} hrs)")
    if input_data["Humidity"] < 20 or input_data["Humidity"] > 80:
        details.append(f"💧 Abnormal Humidity ({input_data['Humidity']}%)")
    if input_data["RPM"] < 500 or input_data["RPM"] > 5000:
        details.append(f"⚡ Abnormal RPM ({input_data['RPM']} RPM)")
    return details

# ตัวอย่างข้อมูลใหม่จากเซ็นเซอร์
new_data = {
    "Temperature": 1,    # ค่าอุณหภูมิ
    "Vibration": 1,        # ค่าการสั่นสะเทือน
    "Machine_Age": 1,        # อายุเครื่องจักร (ปี)
    "Humidity": 1,          # ความชื้น (%)
    "RPM": 1,              # ความเร็วรอบ (RPM)
    "Operating_Hours": 1  # ชั่วโมงการทำงาน
}

# แปลงข้อมูลใหม่เป็น DataFrame
new_df = pd.DataFrame.from_dict([new_data])

# ทำนายผล
predicted_numeric = loaded_model.predict(new_df)
predicted_abnormality = loaded_le.inverse_transform(predicted_numeric)

# แสดงผลลัพธ์
abnormal_details = get_abnormal_details(new_data)
result = {
    "prediction": predicted_abnormality[0],
    "abnormal_details": abnormal_details
}

print(json.dumps(result, indent=4, ensure_ascii=False))

✅ โหลดโมเดลสำเร็จ
{
    "prediction": "Normal",
    "abnormal_details": [
        "💧 Abnormal Humidity (1%)",
        "⚡ Abnormal RPM (1 RPM)"
    ]
}
